In [1]:
# Player Info

import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv


In [3]:
df_match = pd.read_csv("./Data/all_match_info.csv")
match = []
match.append(list(df_match.columns))
match.extend(list(df_match.values))



In [5]:
links = list(pd.read_csv("./Data/all_match_info.csv")['Match Link'])
links[0]

'https://www.espncricinfo.com/series/prudential-world-cup-1975-60793/australia-vs-west-indies-final-65049/full-scorecard'

In [7]:
batting_main_dic ={}
batting_num = 0
bowling_main_dic ={}
bowling_num = 0

batting_player_link = []
batting_player_name = []
batting_players_remove_link = []

bowling_player_link = []
bowler_name = []

matchs = []

c = 1
for link in links[0:]:
    code = requests.get(link)
    soup = BeautifulSoup(code.content,"html.parser")
    table = soup.find_all(attrs={"class":"ds-mt-2"})
    
    for team in table:
        """for i in team.select("span.ds-text-title-xs.ds-font-bold.ds-capitalize"):
            teaminnings = i.text"""

        for tea in team.find_all("table"):
            teaminnings = team.select("span.ds-text-title-xs.ds-font-bold.ds-capitalize")[0].text
            if "ci-scorecard-table" in tea["class"]:
                head = []
                count = 0
                bat = 0
                for i in tea.thead.find_all("th"):
                    head.append(i.text)
                    count+=1
                    if count == 2:
                        head.insert(2,"Batting Possition")
                        head.insert(0,"VS")
                        head.append("Match Id")
                        head.append("Teaminnings")
                        
                body = []
                
                for i in tea.tbody.find_all("tr"):
                    in_body = []
                    count = 0
                    
                    
                    for j in i.find_all("td"):
                        in_body.append(j.text)
                        
                        count+=1
                        if count == 2:
                            bat +=1
                            in_body.insert(2,bat)
                            in_body.insert(0,match[c][0])
                            in_body.append(match[c][7])
                            in_body.append(teaminnings)
                            
                        if j.a:
                            batting_player_name.append(j.a.text)
                            batting_player_link.append(j.a["href"])
                        try:
                            if j.div.strong.text == "Did not bat: ":
                                batting_players_remove_link.append(j.a["href"])
                        except Exception as e:
                            pass                    

                    if len(in_body) == 12:
                        body.append(in_body)
                    
                for h in body:
                    dic = {}
                    for i in range(12):
                        dic[head[i]]=h[i]
                    batting_main_dic[batting_num] = dic
                    batting_num+=1
         
            else:
                head = []
                count = 0
                bat = 0
                for i in tea.thead.find_all("th"):
                    head.append(i.text)
                    count+=1
                    if count == 2:
                        head.insert(0,"VS")
                        head.insert(1,"Match Id")
                        head.append("Teaminnings")
                        
                body = []
                for i in tea.tbody.find_all("tr"):
                    in_body = []
                    count = 0
                    for j in i.find_all("td"):
                        in_body.append(j.text)
                        count+=1
                        if count == 2:
                            in_body.insert(0,match[c][0])
                            in_body.insert(1,match[c][7])
                            in_body.append(teaminnings)
                            
                        if j.a :
                            bowler_name.append(j.a.text)
                            bowling_player_link.append(j.a["href"])
                            
                    
                    if len(in_body) == 14:
                        body.append(in_body)
                    
                for h in body:
                    dic = {}
                    for i in range(14):
                        dic[head[i]]=h[i]
                    bowling_main_dic[bowling_num] = dic
                    bowling_num+=1
    c+=1

In [9]:
team.select("span.ds-text-title-xs.ds-font-bold.ds-capitalize")[0].text

'New Zealand'

In [11]:
team.select("span.ds-text-title-xs.ds-font-bold.ds-capitalize")

[<span class="ds-text-title-xs ds-font-bold ds-capitalize">New Zealand</span>]

In [13]:
bowling = pd.DataFrame(bowling_main_dic.values(),index=list(range(bowling_num)))

In [15]:
# Getting all bowlers link
count = 0
main_links = {}
for i,j in zip(bowler_name,bowling_player_link):
    if i == bowling_main_dic[count]['Bowling']:
        main_links[i] = "https://www.espncricinfo.com" + j
        count+=1



In [17]:
# Getting all batters link
count = 0
for i,j in zip(batting_player_name,batting_player_link):
    if i == bowling_main_dic[count]['Bowling']:
        main_links[i] = "https://www.espncricinfo.com" + j
        count+=1



In [19]:
key = main_links.keys()
value = main_links.values()
all_link_dic = {}
all_link_dic["Name"] = key
all_link_dic["Link"] = value
all_link_dic = pd.DataFrame(all_link_dic)
all_link_dic.to_csv("./Data/Player Links.csv",index=False)

In [21]:

batting = pd.DataFrame(batting_main_dic.values(),index=list(range(batting_num)))

In [23]:
batting.to_csv("./Data/all_t20_wc_batting.csv",index=False)
bowling.to_csv("./Data/all_t20_wc_bowling.csv",index=False)